In [2]:
import os
import tensorflow_datasets as tfds
import tensorflow as tf

import pathlib

In [3]:

_CITATION = """\
@ONLINE {tfflowers,
author = "The TensorFlow Team",
title = "Flowers",
month = "jan",
year = "2019",
url = "http://download.tensorflow.org/example_images/flower_photos.tgz" }
"""

In [4]:
_URL = "http://download.tensorflow.org/example_images/flower_photos.tgz"

In [5]:
class TFFlowers(tfds.core.GeneratorBasedBuilder):
  """Flowers dataset."""

  VERSION = tfds.core.Version("3.0.1")

  def _info(self):
    return tfds.core.DatasetInfo(
        builder=self,
        description="A large set of images of flowers",
        features=tfds.features.FeaturesDict({
            "image":
                tfds.features.Image(),
            "label":
                tfds.features.ClassLabel(names=[
                    "dandelion", "daisy", "tulips", "sunflowers", "roses"
                ]),
        }),
        supervised_keys=("image", "label"),
        homepage="https://www.tensorflow.org/tutorials/load_data/images",
        citation=_CITATION)

  def _split_generators(self, dl_manager):
        
    data_dir = tf.keras.utils.get_file(origin=_URL,
                                       fname='flower_photos',
                                       untar=True,
                                       cache_dir = "tensorflow_datasets")
    data_dir = pathlib.Path(data_dir)
        
    #path_download = dl_manager.download(_URL)
    #ExPth = "tensorflow_datasets\downloads\downl.tenso.org_examp_image_flowe_photoTFSs55Ear_4To2XDT2UOcd1b8b4KWLRk5acYPj5ZXZw.tgz"
    #path_extract = dl_manager.extract(ExPth)
    
    data_dir = pathlib.Path("tensorflow_datasets/datasets/flower_photos")
    image_count = len(list(data_dir.glob('*/*.jpg')))
    print(image_count," images downloaded!")

    
    batch_size = 32
    img_height = 180
    img_width = 180
    
    train_ds = tf.keras.utils.image_dataset_from_directory(
      data_dir,
      validation_split=0.2,
      subset="training",
      seed=123,
      image_size=(img_height, img_width),
      batch_size=batch_size)
    
    val_ds = tf.keras.utils.image_dataset_from_directory(
      data_dir,
      validation_split=0.2,
      subset="validation",
      seed=123,
      image_size=(img_height, img_width),
      batch_size=batch_size)
    
    return [train_ds,val_ds]

  def _generate_examples(self, images_dir_path):
    """Generate flower images and labels given the image directory path.
    Args:
      images_dir_path: path to the directory where the images are stored.
    Yields:
      The image path and its corresponding label.
    """
    for fname, fobj in images_dir_path:
      if fname.endswith(".jpg"):
        image_dir, image_file = os.path.split(fname)
        d = os.path.basename(image_dir)
        record = {"image": fobj, "label": d.lower()}
        yield "%s/%s" % (d, image_file), record

In [6]:
ClassObject = TFFlowers()

In [7]:
ClassObject._info()

tfds.core.DatasetInfo(
    name='tf_flowers',
    full_name='tf_flowers/3.0.1',
    description="""
    A large set of images of flowers
    """,
    homepage='https://www.tensorflow.org/tutorials/load_data/images',
    data_path='C:\\Users\\aleks\\tensorflow_datasets\\tf_flowers\\3.0.1',
    download_size=Unknown size,
    dataset_size=Unknown size,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=5),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
    },
    citation="""@ONLINE {tfflowers,
    author = "The TensorFlow Team",
    title = "Flowers",
    month = "jan",
    year = "2019",
    url = "http://download.tensorflow.org/example_images/flower_photos.tgz" }""",
)

In [8]:
import tensorflow_datasets.core.download.download_manager as dl_manager
manager = dl_manager.DownloadManager(download_dir='tensorflow_datasets\datasets')

In [9]:
DataSets = ClassObject._split_generators(manager)

3670  images downloaded!
Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [10]:
train_ds = DataSets[0]
val_ds = DataSets[1]

In [11]:
class_names = train_ds.class_names
print(class_names)

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [12]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [13]:
import numpy as np
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [14]:
num_classes = 5

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [15]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [16]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

Epoch 1/10
92/92 [==============================] - 46s 482ms/step - loss: 5.9174 - accuracy: 0.4138 - val_loss: 1.1637 - val_accuracy: 0.5422
Epoch 2/10
92/92 [==============================] - 44s 476ms/step - loss: 0.9798 - accuracy: 0.6270 - val_loss: 1.0749 - val_accuracy: 0.5886
Epoch 3/10
92/92 [==============================] - 44s 475ms/step - loss: 0.6236 - accuracy: 0.7922 - val_loss: 1.2911 - val_accuracy: 0.5395
Epoch 4/10
92/92 [==============================] - 43s 470ms/step - loss: 0.3448 - accuracy: 0.9050 - val_loss: 1.2459 - val_accuracy: 0.5763
Epoch 5/10
92/92 [==============================] - 43s 469ms/step - loss: 0.1889 - accuracy: 0.9584 - val_loss: 1.3185 - val_accuracy: 0.5613
Epoch 6/10
92/92 [==============================] - 43s 469ms/step - loss: 0.1085 - accuracy: 0.9785 - val_loss: 1.3286 - val_accuracy: 0.5586
Epoch 7/10
92/92 [==============================] - 44s 478ms/step - loss: 0.0514 - accuracy: 0.9932 - val_loss: 1.5393 - val_accuracy: 0.5640